# Chapter 12 - 내장 자료형 상속과 다중 상속

## 내장 자료형의 상속은 까다롭다

내장 자료형은 '슈퍼클래스에서 구현된 메서드 안에서 호출하더라도 메서드 검색은 대상 객체(self)의 클래스에서 시작해야 한다'는 객체지향 프로그래밍의 기본 규칙을 어기고 있다. 이런 와중에도 \_\_missing\_\_() 메서드는 예외적인 상황을 처리하는 메서드이므로 문서화된 대로 작동한다.

In [1]:
class DoppelDict(dict):
    def __setitem__(self, key, value):
        super().__setitem__(key, [value] * 2)
        
dd = DoppelDict(one=1)
dd

{'one': 1}

In [3]:
dd['two'] = 2
dd

{'one': 1, 'two': [2, 2]}

In [5]:
dd.update(three=3)
dd

{'one': 1, 'two': [2, 2], 'three': 3}

In [7]:
class AnswerDict(dict):
    def __getitem__(self, key):
        return 42
    
ad = AnswerDict(a='foo')
ad['a']

42

In [8]:
d = {}
d.update(ad)
d['a']

'foo'

In [9]:
d

{'a': 'foo'}

dict 대신 collections.UserDict를 상속하면 위에서 발생한 문제가 해결된다.

## 다중 상속과 메서드 결정 순서

In [2]:
class A:
    def ping(self):
        print('ping:', self)
    
class B(A):
    def pong(self):
        print('pong:', self)

class C(A):
    def pong(self):
        print('PONG:', self)
        
class D(B, C):
    def ping(self):
        super().ping()
        print('post-ping:', self)
        
    def pingpong(self):
        self.ping()
        super().ping()
        self.pong()
        super().pong()
        C.pong(self)

In [3]:
d = D()
d.pong()

pong: <__main__.D object at 0x7fdab8123650>


In [5]:
C.pong(d)

PONG: <__main__.D object at 0x7fdab8123650>


In [6]:
D.__mro__

(__main__.D, __main__.B, __main__.C, __main__.A, object)

In [8]:
d.pingpong()

ping: <__main__.D object at 0x7fdab8123650>
post-ping: <__main__.D object at 0x7fdab8123650>
ping: <__main__.D object at 0x7fdab8123650>
pong: <__main__.D object at 0x7fdab8123650>
pong: <__main__.D object at 0x7fdab8123650>
PONG: <__main__.D object at 0x7fdab8123650>


## 실세계에서의 다중 상속

디자인 패턴에서 설명하는 어댑터 패턴이 다중 상속을 사용하므로, 분명 다중 상속이 잘못된 것은 아니다.<br>
물론 그 책에서 설명하는 나머지 22개 패턴은 단일 상속을 사용하므로, 다중 상속이 만병통치약은 아니다.

## 다중 상속 다루기

1. 인터페이스 상속과 구현 상속을 구분한다.
2. ABC를 이용해서 인터페이스를 명확히 한다.
3. 코드를 재사용하기 위해 믹스인을 사용한다.
4. 이름을 통해 믹스인임을 명확히 한다.
5. ABC가 믹스인이 될 수는 있지만, 믹스인이라고 해서 ABC인 것은 아니다.
6. 두 개 이상의 구상 클래스에서 상속받지 않는다.
7. 사용자에게 집합 클래스를 제공한다.
8. 클래스 상속보다 객체 구성을 사용한다.